In [1]:
import numpy as np
import pandas as pd

# Pandas utility functions

Similarly to numpy, pandas also has a few utility functions to identify and detect null values:

In [5]:
pd.isnull(np.nan)

True

In [6]:
pd.isnull(None)

True

In [7]:
pd.isna(None)

True

The opposite ones also exist:

In [8]:
pd.notnull(None)

False

In [9]:
pd.notna(None)

False

In [10]:
pd.notnull(3)

True

In [12]:
pd.isnull(pd.Series([1,5, None , np.nan,  6]))

0    False
1    False
2     True
3     True
4    False
dtype: bool

In [13]:
pd.notnull(pd.Series([1,5, None , np.nan,  6]))

0     True
1     True
2    False
3    False
4     True
dtype: bool

In [14]:
pd.isnull(pd.DataFrame({'Column A': [1, np.nan, 8],
                       'Columan B': [5, None, 9],
                       'Column C': [np.nan, 5, np.nan]}))


,Column A,Columan B,Column C
0,False,False,True
1,True,True,False
2,False,False,True


# Pandas Operations with Missing Values


Pandas manages missing values more gracefully than numpy. nans will no longer behave as "viruses", and operations will just ignore them completely:

In [15]:
pd.Series([1, 2, np.nan]).count()

2

In [16]:
pd.Series([1, 2, np.nan]).sum()

3.0

In [17]:
pd.Series([2, 2, np.nan]).mean()

2.0

# Filtering missing data
As we saw with numpy, we could combine boolean selection + pd.isnull to filter out those nans and null values:

In [36]:
s = pd.Series([1, 2, 3, np.nan, np.nan, None,  5, 4])


In [37]:
pd.notnull(s)

0     True
1     True
2     True
3    False
4    False
5    False
6     True
7     True
dtype: bool

In [38]:
pd.notnull(s).mean()

0.625

In [39]:
pd.notnull(s).sum() #Gives number of non-null values in series

5

In [40]:
pd.isnull(s).sum() # gives number of nan value in series

3

In [41]:
s[pd.notnull(s)]

0    1.0
1    2.0
2    3.0
6    5.0
7    4.0
dtype: float64

But both **notnull** and **isnull** are also methods of Series and DataFrames, so we could use it that way:

In [42]:
s.isnull()

0    False
1    False
2    False
3     True
4     True
5     True
6    False
7    False
dtype: bool

In [43]:
s.notnull()

0     True
1     True
2     True
3    False
4    False
5    False
6     True
7     True
dtype: bool

In [44]:
s[s.isnull()]

3   NaN
4   NaN
5   NaN
dtype: float64

In [45]:
s[s.notnull()]

0    1.0
1    2.0
2    3.0
6    5.0
7    4.0
dtype: float64

# Dropping null values
Boolean **selection + notnull()** seems a little bit verbose and repetitive. And as we said before: any repetitive task will probably have a better, more DRY way. In this case, we can use the **dropna** method:

In [46]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    NaN
6    5.0
7    4.0
dtype: float64

In [47]:
s.dropna()

0    1.0
1    2.0
2    3.0
6    5.0
7    4.0
dtype: float64

# Dropping null values on DataFrames
You saw how simple it is to drop **nas** with a Series. But with DataFrames, there will be a few more things to consider, because you can't drop single values. You can only drop entire columns or rows. Let's start with a sample DataFrame:

In [48]:
df = pd.DataFrame({
    'Column A': [1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110],
})

In [49]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [50]:
df.shape

(4, 4)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Column A  2 non-null      float64
 1   Column B  3 non-null      float64
 2   Column C  3 non-null      float64
 3   Column D  4 non-null      int64  
dtypes: float64(3), int64(1)
memory usage: 256.0 bytes


In [53]:
df.isnull().sum()

Column A    2
Column B    1
Column C    1
Column D    0
dtype: int64

The default dropna behavior will drop all the rows in which any null value is present:

In [54]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In [55]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In this case we're dropping rows. Rows containing null values are dropped from the DF. You can also use the axis parameter to drop columns containing null values:

In [56]:
df.dropna(axis=1) # axis='columns' also works

,Column D
0,5
1,8
2,34
3,110


In this case, any row or column that contains at least one null value will be dropped. Which can be, depending on the case, too extreme. You can control this behavior with the **how** parameter. Can be either **'any' or 'all'**:

In [61]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [58]:
df.dropna(how= 'all')

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [59]:
df.dropna(how='any')

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


You can also use the thresh parameter to indicate a threshold (a minimum number) of non-null values for the row/column to be kept:

In [60]:
df.dropna(thresh= 3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [62]:
df.dropna(thresh=3, axis='columns')

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


# Filling null values
Sometimes instead than dropping the null values, we might need to replace them with some other value. This highly depends on your context and the dataset you're currently working. Sometimes a nan can be replaced with a 0, sometimes it can be replaced with the mean of the sample, and some other times you can take the closest value. Again, it depends on the context. We'll show you the different methods and mechanisms and you can then apply them to your own problem.

In [63]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    NaN
6    5.0
7    4.0
dtype: float64

### Filling nulls with a arbitrary value

In [64]:
s.fillna(0)

0    1.0
1    2.0
2    3.0
3    0.0
4    0.0
5    0.0
6    5.0
7    4.0
dtype: float64

In [65]:
s.fillna(s.mean())

0    1.0
1    2.0
2    3.0
3    3.0
4    3.0
5    3.0
6    5.0
7    4.0
dtype: float64

In [66]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    NaN
6    5.0
7    4.0
dtype: float64

### Filling nulls with contiguous (close) values
The method argument is used to fill null values with other values close to that null one:

In [68]:
s.fillna(method='ffill')

0    1.0
1    2.0
2    3.0
3    3.0
4    3.0
5    3.0
6    5.0
7    4.0
dtype: float64

In [69]:
s.fillna(method='bfill')

0    1.0
1    2.0
2    3.0
3    5.0
4    5.0
5    5.0
6    5.0
7    4.0
dtype: float64

This can still leave null values at the extremes of the Series/DataFrame:



In [70]:
pd.Series([np.nan, 3, np.nan, 9]).fillna(method='ffill')

0    NaN
1    3.0
2    3.0
3    9.0
dtype: float64

# Filling null values on DataFrames
The fillna method also works on DataFrames, and it works similarly. The main differences are that you can specify the axis (as usual, rows or columns) to use to fill the values (specially for methods) and that you have more control on the values passed:

In [71]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [72]:
df.fillna({'Column A': 0, 'Column B': 99, 'Column C': df['Column C'].mean()})


,Column A,Column B,Column C,Column D
0,1.0,2.0,47.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,99.0,100.0,110


In [75]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [73]:
df.fillna(method='ffill', axis=0)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [74]:

df.fillna(method='ffill', axis=1)

,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0
